In [1]:
import pandas as pd
import numpy as np

In [2]:
orf_metadata_df = pd.read_csv('output/orf_metadata.tsv.gz', sep='\t')
orf_metadata_df.head()

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,...,Metadata_disease_involvement,Metadata_subcellular_location,Metadata_gene_group_id,Metadata_homologue,Metadata_biological_process,Metadata_cellular_component,Metadata_molecular_function,Metadata_wikipathway,Metadata_dependency_probability,Metadata_TPM
0,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,...,NaN,NaN,1133,True,GOBP_CELLULAR_RESPONSE_TO_XENOBIOTIC_STIMULUS|...,NaN,GOMF_ACETYLTRANSFERASE_ACTIVITY|GOMF_ACYLTRANS...,NaN,0.019718,2.028569
1,JCP2022_900003,ccsbBroad304_00002,ORF005388.1_TRC304.1,pLX_304,NM_001088.3,AANAT,15,9606,aralkylamine N-acetyltransferase,100.0,...,Disease variant,Cytosol,1134,True,GOBP_AMIDE_BIOSYNTHETIC_PROCESS|GOBP_AMIDE_MET...,GOCC_PERINUCLEAR_REGION_OF_CYTOPLASM,GOMF_14_3_3_PROTEIN_BINDING|GOMF_ACETYLTRANSFE...,WP_BIOGENIC_AMINE_SYNTHESIS|WP_BIOSYNTHESIS_AN...,0.022395,0.400538
2,JCP2022_900004,ccsbBroad304_00003,ORF003876.1_TRC304.1,pLX_304,NM_000663.4,ABAT,18,9606,4-aminobutyrate aminotransferase,100.0,...,Disease variant|FDA approved drug targets,Mitochondria,NaN,True,GOBP_ACIDIC_AMINO_ACID_TRANSPORT|GOBP_ACID_SEC...,GOCC_CATALYTIC_COMPLEX|GOCC_INTRACELLULAR_PROT...,GOMF_METAL_CLUSTER_BINDING|GOMF_TRANSFERASE_AC...,WP_ALANINE_AND_ASPARTATE_METABOLISM|WP_FRAGILE...,0.003862,3.109361
3,JCP2022_900005,ccsbBroad304_00007,ORF004679.1_TRC304.1,pLX_304,NM_000018.4,ACADVL,37,9606,acyl-CoA dehydrogenase very long chain,100.0,...,Cardiomyopathy|Disease variant,Mitochondria,2258|974,True,GOBP_ALCOHOL_METABOLIC_PROCESS|GOBP_CELLULAR_K...,GOCC_ENVELOPE|GOCC_EXTRINSIC_COMPONENT_OF_MEMB...,GOMF_ACYL_COA_BINDING|GOMF_ADENYL_NUCLEOTIDE_B...,WP_FATTY_ACID_BETA_OXIDATION|WP_MITOCHONDRIAL_...,0.001691,7.987150
4,JCP2022_900006,ccsbBroad304_00008,ORF000425.1_TRC304.1,pLX_304,NM_001095.4,ASIC1,41,9606,acid sensing ion channel subunit 1,100.0,...,NaN,Golgi apparatus|Plasma membrane,290,True,GOBP_ASSOCIATIVE_LEARNING|GOBP_BEHAVIOR|GOBP_B...,GOCC_CELL_SURFACE|GOCC_GOLGI_APPARATUS|GOCC_PR...,GOMF_GATED_CHANNEL_ACTIVITY|GOMF_INORGANIC_MOL...,NaN,0.215086,4.014355


In [3]:
orf_metadata_df = orf_metadata_df.assign(
    Metadata_Gene_Symbol=lambda x: np.where(
        x.Metadata_Approved_Symbol.notna(),
        x.Metadata_Approved_Symbol,
        x.Metadata_Symbol,
    )
)

Strategy for removing duplicates
- Keep the one with the longest insert
- Keep the one with the highest number of proteins matching
- If multiple ones have the same number of protein matching and the same insert length, keep the first one in the sorted list

In [4]:
orf_metadata_deduplicated_df = (
    orf_metadata_df.sort_values(
        by=["Metadata_Insert_Length", "Metadata_Prot_Match"], ascending=False
    )
    .drop_duplicates("Metadata_Gene_Symbol", keep="first")
    .sort_values(by="Metadata_JCP2022")
    .drop(columns=["Metadata_Gene_Symbol"])
    .reset_index(drop=True)
)

In [5]:
orf_metadata_deduplicated_df.to_csv("output/orf_metadata_deduplicated.tsv.gz", sep="\t", index=False)